### Multinomial Classification with Logistic Regression of Scikit Learn v/s Minimizing Logistic cost function using Powell's conjugate direction optimization method

#### Import statements

In [2]:
import numpy as np
from __future__ import division
import pandas as pd
import math
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

#### Import dataset from file, separate features and labels

In [3]:
data = pd.read_csv('wine.data', header = None)
num_features = data.shape[1]-1
num_examples = data.shape[0]
print "Number of features: ",num_features
print "Number of examples: ",num_examples
data.head()

Number of features:  13
Number of examples:  178


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
Y = data[0]
print Y.shape
Y.head()

(178L,)


0    1
1    1
2    1
3    1
4    1
Name: 0, dtype: int64

In [5]:
X = data[range(1,num_features+1)]
print X.shape
X = preprocessing.scale(X)
X = pd.DataFrame(X)
X.head()
# print X[:5]

(178, 13)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009
1,0.246290,-0.499413,-0.827996,-2.490847,0.018145,0.568648,0.733629,-0.820719,-0.544721,-0.293321,0.406051,1.113449,0.965242
2,0.196879,0.021231,1.109334,-0.268738,0.088358,0.808997,1.215533,-0.498407,2.135968,0.269020,0.318304,0.788587,1.395148
3,1.691550,-0.346811,0.487926,-0.809251,0.930918,2.491446,1.466525,-0.981875,1.032155,1.186068,-0.427544,1.184071,2.334574
4,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874


#### Partition data into training and test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(
                                    X, Y, train_size = 128, random_state=42)
num_train_examples = X_train.shape[0]
print "Number of training examples: ", num_train_examples
num_test_examples = X_test.shape[0]
print "Number of test examples: ", num_test_examples
X_train.head()
Y_train.head()
# print X_train.head()
# print Y_train.head()
# print X_train[:5]

Number of training examples:  128
Number of test examples:  50


76     2
56     1
26     1
153    3
138    3
Name: 0, dtype: int64

#### Perform vanilla multinomial Logistic regression

In [8]:
from sklearn.linear_model import LogisticRegression
LR_clf = LogisticRegression(solver = 'newton-cg', multi_class ='multinomial', max_iter = 1000, C=9223372036854775807)
LR_clf.fit(X_train, Y_train)
Y_prediction = LR_clf.predict(X_test)
accuracy = accuracy_score(Y_test, Y_prediction)
print accuracy

0.98


#### Compute log loss using scikit-learn

In [9]:
LR_clf_prob = LR_clf.predict_proba(X_train)
print LR_clf_prob[:5]
from sklearn.metrics import log_loss
LR_clf_log_loss = log_loss(Y_train, LR_clf_prob)
print LR_clf_log_loss

[[  4.16366443e-14   1.00000000e+00   3.40145475e-18]
 [  1.00000000e+00   3.43479045e-17   1.79448295e-11]
 [  9.99999994e-01   1.17779393e-17   6.48390844e-09]
 [  1.29929414e-09   1.14150142e-16   9.99999999e-01]
 [  5.72434298e-08   2.75435727e-07   9.99999667e-01]]
2.03940579736e-06


#### Create empty weight vector of 42-dimension for coordinate descent

In [10]:
weight = np.zeros(42)
print X.shape

(178, 13)


#### Modify feature vector by adding extra dimension of all 1s

In [12]:
# offset = np.empty(178)[...,None]
offset = np.empty(178)
offset.fill(1)
# X = np.append(X, offset, 1)
X_train[14] = pd.Series(offset)
X_test[14] = pd.Series(offset)
print X_train.shape
print X_test.shape

(128, 14)
(50, 14)


C:\Users\Rahul\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Rahul\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
# z = np.zeros(14)
# z.fill(1)
# X = preprocessing.scale(X)
# X_train, X_test, Y_train, Y_test = train_test_split(
#                                     X, Y, train_size = 128, random_state=42)
X_train_matrix = X_train.as_matrix()
# X_train_matrix = preprocessing.scale(X_train_matrix)
Y_train_matrix = Y_train.as_matrix()
# print np.dot(X_train_matrix[0],z)



# X_train_matrix = X_train
# Y_train_matrix = Y_train
print X_train_matrix.shape
print Y_train_matrix.shape
# print Y_train_matrix[105]

(128L, 14L)
(128L,)


#### Function to compute cost function

In [14]:
# Works when everything is numpy array
def cost_function(weight):
    result = 0
    # for each training example
    for i in range(num_train_examples):
        
        # declaring all zero indicator function for every new iteration
        indicator = [0,0,0]
        # extracting y(i)
        y_i = Y_train_matrix[i]
        # initializing indicator function appropriately
        indicator[y_i-1]=1
        
        log_term = 0
        
        # to compute log of summation term
        for j in range(3):
            log_term += math.exp( np.dot( weight[j*14:(j+1)*14],X_train_matrix[i]) )
        
        # compute w dot x; taking only that w vector which is non zero, using y_i    
        dot_product = np.dot(weight[(y_i-1)*14:(y_i)*14], X_train_matrix[i])
        
        result+= (dot_product - log_term)
    result = -1.0*result
    result = result/num_train_examples
    return result

In [26]:
from scipy.optimize import fmin, fmin_powell
weight_opt = fmin_powell(cost_function, weight)
print weight_opt

Optimization terminated successfully.
         Current function value: 1.201249
         Iterations: 32
         Function evaluations: 14890
[ 0.36838478  0.01137444  0.37656687 -0.37619079  0.13443355 -0.41263159
  1.0222516  -0.26813298 -0.2555348  -0.33890556  0.03870316  0.31184334
  0.46678638 -2.41802737 -0.24806748 -0.17572123 -0.29142958  0.3251041
  0.14830956 -0.3321599   0.71420175 -0.07866364 -0.06103103 -1.19170129
 -0.05287258 -0.14848795 -0.53605258 -2.05704519 -0.03713139 -0.02311029
  0.29927831 -0.03770555  0.15514133  0.18662674 -1.75253634 -0.14153262
 -0.31679453  0.31092998 -0.11071838 -0.95543551 -0.0965426  -4.05299586]


In [27]:
weight_vectors = np.split(weight_opt, 3)
print weight_vectors[2]

[-0.03713139 -0.02311029  0.29927831 -0.03770555  0.15514133  0.18662674
 -1.75253634 -0.14153262 -0.31679453  0.31092998 -0.11071838 -0.95543551
 -0.0965426  -4.05299586]


In [28]:
X_test_matrix = X_test.as_matrix()
Y_test_predict = np.empty(num_test_examples, dtype=int)
for i in range(num_test_examples):
    c1 = np.dot(weight_vectors[0], X_test_matrix[i])
    c2 = np.dot(weight_vectors[1], X_test_matrix[i])
    c3 = np.dot(weight_vectors[2], X_test_matrix[i])
    c_final = np.argmax([c1, c2, c3]) +1
    Y_test_predict[i] = int(c_final)

In [29]:
print Y_test_predict

[1 1 3 1 2 1 2 3 2 3 1 3 1 2 1 2 2 2 1 2 1 2 3 3 3 3 2 2 2 1 1 2 3 1 1 1 3
 3 2 3 1 2 2 3 3 1 2 2 3 1]


In [30]:
print accuracy_score(Y_test, Y_test_predict)

0.96
